## Imports, global vars, functions

In [1]:
import csv
import re
import string
from timeit import default_timer as timer
import gensim
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Dropout, Embedding, Input, Dropout, Bidirectional, GaussianNoise
from keras.layers.recurrent import LSTM, GRU

import matplotlib.pyplot as plt
import pandas as pd

/home/Liam/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Just to check if GPU is working as intended
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
NOTEEVENTS = "./data/NOTEEVENTS.csv"
DIAGNOSES_ICD = "./data/DIAGNOSES_ICD.csv"

# record the top codes in a list
ICD_CODES = ["4019","4280","42731", "41401", "5849"]

In [4]:
def clean_string1(text):
    """
    """
    text = text.strip().lower().replace('-', '_').replace('.', '_').replace(' ', '_').rstrip('_')
    return text

def preprocess_text2(query):
    """
    """
    query = re.sub('\d+|\d+\.\d+', '[NUM]', query)
    query = re.sub('(\[\*\*.*?\*\*\])', '[PHI]', query)
    query = query.strip('"').strip('?').strip("'").strip('(').strip(')').strip(':')
    query = re.sub('['+'!"#$%&\'()*+,-./:;<=>?@\\^`{|}~'+']', '', query)
    word_list = query.split()
    word_list = [clean_string1(word) for word in word_list]
    return word_list

## Fetch embeddings and data

### Embeddings

In [5]:
word_vectors = Word2Vec.load('./data/w2v_embeddings')
# word_vectors = FastText.load('fasttext_embeddings')
embedding_dim = word_vectors.wv.vectors.shape[1]

### Input data [IGNORE - SEE SECTION BELOW]

In [9]:
'''
**ONLY RUN IF NEED TO RE-PROCESS TEXT**
Read in notes and pre-process text
Also fetch HADM_IDs
'''
hadm_id = []
input_notes = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        if note[6] == 'Discharge summary':
            hadm_id.append(note[2])
            note = preprocess_text2(note[-1])
            input_notes.append(note)

In [13]:
'''
Save processed input notes
'''
with open("processed_input_notes.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(input_notes)

In [110]:
'''
Only fetch HADM_IDs
'''
hadm_id = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        if note[6] == 'Discharge summary':
            hadm_id.append(note[2])

In [113]:
'''
Load processed input notes 
'''
input_notes = []
with open("processed_input_notes.csv", "r") as f:
    reader = csv.reader(f)
    for note in reader:
        input_notes.append(note)
deleted_dupes = 0

### Labels [AND INPUT PROCESSING]

In [6]:
df1 = pd.read_csv(DIAGNOSES_ICD)
df2 = pd.read_csv(NOTEEVENTS)

/home/Liam/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# Translate ICD codes to indicator variables
dummy = pd.get_dummies(df1['ICD9_CODE'])[ICD_CODES]
# Append indicator var columnns to original ICD df
dummy_combined = pd.concat([df1, dummy], axis=1)
# Combine by HADM_ID and drop columns
dummy_combined = dummy_combined.groupby(['HADM_ID'], as_index=False).sum().drop(['ROW_ID','SUBJECT_ID','SEQ_NUM'], axis = 1)

#now join the two tables together 
df_final = pd.merge(df2, dummy_combined,left_on="HADM_ID", right_on='HADM_ID', how='left')
# Filter by discharge summary
df_final = df_final[df_final['CATEGORY'] == 'Discharge summary']
# removed any hadmid that have more than one entry in database
df_final = df_final.drop_duplicates(subset = "HADM_ID", keep = False)
df_final = df_final.drop(['ROW_ID', 'SUBJECT_ID', 'CHARTDATE', 
                          'CHARTTIME', 'STORETIME', 'CATEGORY',
                         'DESCRIPTION', 'CGID', 'ISERROR'], axis = 1)

In [18]:
#random sample of the data
sub_df_final = df_final.sample(10000)

In [19]:
input_notes = []
hadm_id = []
y = []
for i in range(len(sub_df_final)):
    y.append(list(map(int, sub_df_final.iloc[i, 2:].tolist())))
    input_notes.append(preprocess_text2(sub_df_final.iloc[i, 1]))
    hadm_id.append(sub_df_final.iloc[i, 0])
y = np.array(y)

In [20]:
'''
Train/test split
'''
x_train, x_test, y_train, y_test = train_test_split(input_notes, y, test_size=0.2)

In [21]:
from keras.preprocessing.text import Tokenizer
'''
Takes words and converts to indices (which then correspond to vectors in
the embedding models)
'''
#max_words = len(word_vectors.wv.vocab)
max_words = 15000
token = Tokenizer(max_words)
token.fit_on_texts(input_notes)
vocab_size = max_words + 1

sequences = token.texts_to_sequences(x_train)
test_sequences = token.texts_to_sequences(x_test)

'''
Convert to padded sequences
'''
from keras.preprocessing.sequence import pad_sequences
seq_len = 3000
X = pad_sequences(sequences, maxlen=seq_len)
X_test = pad_sequences(test_sequences, maxlen=seq_len)

embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in word_vectors.wv.vocab:
      coefs = np.asarray(word_vectors.wv[word], dtype='float32')
      embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 62520 word vectors.


In [22]:
word_index = token.word_index
embedding_matrix = np.zeros((vocab_size , embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

In [23]:
num_classes = y_train.shape[1]

## Model building

In [24]:
import keras.backend as K

config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
session = K.tf.Session(config=config)

from keras.layers import Lambda
ClipLayer = Lambda(lambda x: K.clip(x, min_value=0.01, max_value=0.99))

In [ ]:
from keras.optimizers import Adam
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Dropout, Embedding, Input, Dropout, Bidirectional, GaussianNoise
from keras.layers.recurrent import LSTM, GRU

## Build the model ##
input = Input(shape=(seq_len,))
x = Embedding(input_dim = vocab_size , output_dim = embedding_dim, weights=[embedding_matrix], trainable=False)(input)
x = GaussianNoise(0.75)(x)
x = Bidirectional(GRU(units = 128, recurrent_dropout=0.2, dropout=0.2, activation = 'relu', return_sequences=True))(x)
x = Bidirectional(GRU(units = 128, recurrent_dropout=0.2, dropout=0.2, activation = 'relu'))(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='sigmoid')(x)
x = ClipLayer(x)
model = Model(input,x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X,y_train,epochs=5, batch_size = 128) 

Epoch 1/5
3072/8000 [==========>...................] - ETA: 12:56 - loss: 0.6023 - acc: 0.7070